In [1]:
import pandas as pd
import time
from sklearn.metrics import classification_report
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from pathlib import Path
import time
from tqdm import tqdm

In [2]:
# Settings
train_dir = Path('./data/train')
val_dir   = Path('./data/test')
img_size = 48
batch_size = 64

# Transforms
train_transforms = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(5),
    transforms.RandomAffine(degrees=0, shear=10, translate=(0.2, 0.2)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)  # normalize to [-1, 1]
])

val_transforms = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

# Datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transforms)
val_dataset   = datasets.ImageFolder(root=val_dir, transform=val_transforms)
print(f"Train images: {len(train_dataset)}")
print(f"Val images: {len(val_dataset)}")

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# Class names
class_names = train_dataset.classes
print("Classes:", class_names)

Train images: 33614
Val images: 7178
Classes: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


In [3]:
class InvertedResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride, expand_ratio):
        super().__init__()
        self.use_res_connect = stride == 1 and in_channels == out_channels
        hidden_dim = in_channels * expand_ratio

        layers = []

        # Expansion
        if expand_ratio != 1:
            layers.extend([
                nn.Conv2d(in_channels, hidden_dim, kernel_size=1, bias=False),
                nn.BatchNorm2d(hidden_dim),
                nn.ReLU6(inplace=True)
            ])

        # Depthwise
        layers.extend([
            nn.Conv2d(hidden_dim, hidden_dim, kernel_size=3, stride=stride, padding=1, groups=hidden_dim, bias=False),
            nn.BatchNorm2d(hidden_dim),
            nn.ReLU6(inplace=True)
        ])

        # Projection
        layers.extend([
            nn.Conv2d(hidden_dim, out_channels, kernel_size=1, bias=False),
            nn.BatchNorm2d(out_channels)
        ])

        self.block = nn.Sequential(*layers)

    def forward(self, x):
        if self.use_res_connect:
            return x + self.block(x)
        else:
            return self.block(x)

class MobileNetV2_48x48(nn.Module):
    def __init__(self, num_classes=7, dropout_rate=0.0):
        super().__init__()
        self.dropout_rate = dropout_rate
        self.dropout = nn.Dropout(dropout_rate) if dropout_rate > 0 else nn.Identity()

        # Initial convolution layer
        self.init_conv = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1, bias=False),  # 48x48 input
            nn.BatchNorm2d(32),
            nn.ReLU6(inplace=True)
        )

        # Bottleneck configuration: (t, c, n, s)
        bottlenecks_cfg = [
            (1, 16, 1, 1),
            (6, 24, 1, 1),  # no downsampling yet
            (6, 32, 2, 2),
            (6, 64, 1, 2),
            (6, 96, 1, 1),
        ]

        blocks = []
        in_channels = 32
        for t, out_channels, n, s in bottlenecks_cfg:
            for i in range(n):
                stride = s if i == 0 else 1
                blocks.append(InvertedResidualBlock(in_channels, out_channels, stride, t))
                in_channels = out_channels
        self.bottlenecks = nn.Sequential(*blocks)

        self.last_conv = nn.Sequential(
            nn.Conv2d(in_channels, 128, kernel_size=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU6(inplace=True)
        )

        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.init_conv(x)
        x = self.bottlenecks(x)
        x = self.last_conv(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        return self.classifier(x)

In [4]:
# Define MobileNetV2 model
mobilenet_model = MobileNetV2_48x48(num_classes=len(class_names))
total_params = sum(p.numel() for p in mobilenet_model.parameters())
print(f"Total parameters: {total_params}")

Total parameters: 132935


In [5]:
if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")
else:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device: {device}")
mobilenet_model = MobileNetV2_48x48(num_classes=len(class_names)).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mobilenet_model.parameters(), lr=0.001)

Using device: mps


In [6]:
def train(model, train_loader, val_loader, optimizer, criterion, num_epochs=15, model_name='resnet_model.pth'):
    history = []  # collect metrics per epoch

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        start_time = time.time()

        # Training phase
        model.train()
        train_loss, correct, total = 0, 0, 0
        for images, labels in tqdm(train_loader, desc="Training"):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

        train_acc = correct / total
        train_loss /= total

        # Validation phase
        model.eval()
        val_loss, correct, total = 0, 0, 0
        y_true, y_pred = [], []

        with torch.no_grad():
            for images, labels in tqdm(val_loader, desc="Validation"):
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * images.size(0)

                _, predicted = outputs.max(1)
                correct += (predicted == labels).sum().item()
                total += labels.size(0)

                y_true.extend(labels.cpu().numpy())
                y_pred.extend(predicted.cpu().numpy())

        val_acc = correct / total
        val_loss /= total

        # Classification report
        report = classification_report(y_true, y_pred, target_names=class_names, output_dict=True)
        val_f1_macro = report['macro avg']['f1-score']

        epoch_time = time.time() - start_time

        # Collect metrics
        epoch_data = {
            'epoch': epoch + 1,
            'train_loss': train_loss,
            'val_loss': val_loss,
            'train_acc': train_acc,
            'val_acc': val_acc,
            'val_f1_macro': val_f1_macro,
            'epoch_time_sec': epoch_time
        }

        # Add per-class metrics
        for cls in class_names:
            for metric in ['precision', 'recall', 'f1-score']:
                key = f'{cls}_{metric}'
                epoch_data[key] = report[cls][metric]

        history.append(epoch_data)

        # Console output
        print("\nValidation Report:")
        for cls in class_names:
            cls_metrics = report[cls]
            print(f"{cls}: Prec={cls_metrics['precision']:.3f} | Rec={cls_metrics['recall']:.3f} | F1={cls_metrics['f1-score']:.3f}")

        print(f"\nEpoch Summary: Train Loss={train_loss:.4f} | Val Loss={val_loss:.4f} | Train Acc={train_acc:.3f} | Val Acc={val_acc:.3f}")
        print(f"Epoch time: {epoch_time:.2f} seconds")

    torch.save(model.state_dict(), f"results/{model_name}")
    print(f"Saved model to 'results/{model_name}'")

    return history

In [7]:
def build_model(dropout):
    return MobileNetV2_48x48(num_classes=len(class_names), dropout_rate=dropout).to(device)

# Define settings to test
configs = [
    {'name': 'mobv2_adam_lr3_dropout0_bn', 'optimizer': 'adam', 'lr': 1e-3, 'dropout': 0},
    {'name': 'mobv2_adam_lr3_dropout15_bn', 'optimizer': 'adam', 'lr': 1e-3, 'dropout': 0.15},
    {'name': 'mobv2_adam_lr2_dropout15_bn', 'optimizer': 'adam', 'lr': 1e-2, 'dropout': 0.15},
    {'name': 'mobv2_adam_lr3_dropout20_bn', 'optimizer': 'adam', 'lr': 1e-3, 'dropout': 0.20},
    {'name': 'mobv2_sgd_mom09_lr3_dropout15_bn', 'optimizer': 'sgd', 'lr': 1e-3, 'dropout': 0.15},
]


for config in configs:
    print(f"\nTraining config: {config['name']}")

    model = build_model(config['dropout'])
    if config['optimizer'] == 'adam':
        optimizer = optim.Adam(model.parameters(), lr=config['lr'])
    else:
        optimizer = optim.SGD(model.parameters(), lr=config['lr'], momentum=0.9)

    criterion = nn.CrossEntropyLoss()
    history = train(model, train_loader, val_loader, optimizer, criterion, num_epochs=15, model_name=config['name'])
    pd.DataFrame(history).to_csv(f'results/history_{config["name"]}.csv', index=False)


Training config: mobv2_adam_lr3_dropout0_bn

Epoch 1/15


Validation: 100%|██████████| 113/113 [00:20<00:00,  5.52it/s]
/Users/taniapazospuig/Desktop/DeepLearningProject/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/taniapazospuig/Desktop/DeepLearningProject/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/taniapazospuig/Desktop/DeepLearningProject/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no pre


Validation Report:
angry: Prec=0.000 | Rec=0.000 | F1=0.000
disgust: Prec=0.017 | Rec=0.514 | F1=0.032
fear: Prec=0.225 | Rec=0.026 | F1=0.047
happy: Prec=0.343 | Rec=0.397 | F1=0.368
neutral: Prec=0.385 | Rec=0.020 | F1=0.039
sad: Prec=0.296 | Rec=0.053 | F1=0.090
surprise: Prec=0.430 | Rec=0.656 | F1=0.520

Epoch Summary: Train Loss=1.6923 | Val Loss=2.5653 | Train Acc=0.333 | Val Acc=0.198
Epoch time: 157.66 seconds

Epoch 2/15


Validation: 100%|██████████| 113/113 [00:19<00:00,  5.85it/s]



Validation Report:
angry: Prec=0.254 | Rec=0.406 | F1=0.312
disgust: Prec=0.023 | Rec=0.027 | F1=0.025
fear: Prec=0.215 | Rec=0.155 | F1=0.180
happy: Prec=0.580 | Rec=0.762 | F1=0.659
neutral: Prec=0.425 | Rec=0.363 | F1=0.391
sad: Prec=0.365 | Rec=0.218 | F1=0.273
surprise: Prec=0.672 | Rec=0.525 | F1=0.589

Epoch Summary: Train Loss=1.4191 | Val Loss=1.4786 | Train Acc=0.460 | Val Acc=0.426
Epoch time: 144.60 seconds

Epoch 3/15


...


Validation Report:
angry: Prec=0.542 | Rec=0.033 | F1=0.063
disgust: Prec=0.026 | Rec=0.928 | F1=0.052
fear: Prec=0.293 | Rec=0.055 | F1=0.092
happy: Prec=0.929 | Rec=0.220 | F1=0.356
neutral: Prec=0.618 | Rec=0.114 | F1=0.193
sad: Prec=0.480 | Rec=0.029 | F1=0.054
surprise: Prec=0.317 | Rec=0.884 | F1=0.467

Epoch Summary: Train Loss=1.0209 | Val Loss=3.7977 | Train Acc=0.615 | Val Acc=0.208
Epoch time: 158.62 seconds
Saved model to 'results/mobv2_adam_lr3_dropout15_bn'

Training config: mobv2_adam_lr2_dropout15_bn

Epoch 1/15


Validation: 100%|██████████| 113/113 [00:19<00:00,  5.87it/s]
/Users/taniapazospuig/Desktop/DeepLearningProject/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/taniapazospuig/Desktop/DeepLearningProject/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/taniapazospuig/Desktop/DeepLearningProject/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no pre


Validation Report:
angry: Prec=0.000 | Rec=0.000 | F1=0.000
disgust: Prec=0.016 | Rec=0.045 | F1=0.024
fear: Prec=0.000 | Rec=0.000 | F1=0.000
happy: Prec=0.281 | Rec=0.044 | F1=0.076
neutral: Prec=0.194 | Rec=0.039 | F1=0.065
sad: Prec=0.226 | Rec=0.015 | F1=0.029
surprise: Prec=0.129 | Rec=0.974 | F1=0.228

Epoch Summary: Train Loss=1.7457 | Val Loss=3.9129 | Train Acc=0.303 | Val Acc=0.134
Epoch time: 151.72 seconds

Epoch 2/15


Validation:  57%|█████▋    | 64/113 [00:18<00:13,  3.51it/s]


KeyboardInterrupt: 